In [1]:
"""
@description: 执行训练
"""


"""
工程所在绝对路径
"""
# abspath: absolute path to the project
# realpath: canonical path of the specified filename, eliminating any symbolic links encountered in the path 
from os.path import dirname, abspath, realpath, curdir
PROJECT_ROOT = realpath(curdir)
import sys
sys.path.append(PROJECT_ROOT)

"""
import
"""
from config import ConfigTrain  # 导入配置
import utils
from os.path import join as pjoin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch.nn.functional as F
import torch



In [2]:
torch.cuda.current_device()

0

In [3]:
DATA_LIST_ROOT = pjoin(PROJECT_ROOT, 'data_list')

cfg = ConfigTrain()
print('Pick device: ', cfg.DEVICE)
device = torch.device(cfg.DEVICE)

# 网络
print('Generating net: ', cfg.NET_NAME)
net = utils.create_net(3, cfg.NUM_CLASSES, net_name=cfg.NET_NAME)
if cfg.PRETRAIN:  # 加载预训练权重
    print('Load pretrain weights: ', cfg.PRETRAINED_WEIGHTS)
    net.load_state_dict(torch.load(cfg.PRETRAINED_WEIGHTS, map_location='cpu'))
net.to(device)
# 优化器
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.BASE_LR) 


Pick device:  cuda:0
Generating net:  resnext50_32x4d


In [8]:
 # 训练数据生成器
print('Preparing data... batch_size: {}, image_size: {}, crop_offset: {}'.format(cfg.BATCH_SIZE, cfg.IMAGE_SIZE, cfg.HEIGHT_CROP_OFFSET))
df_train = pd.read_csv(pjoin(cfg.DATA_LIST_ROOT, 'train.csv'))
data_generator = utils.train_data_generator(np.array(df_train['image']),
                                            np.array(df_train['label']),
                                            cfg.BATCH_SIZE, cfg.IMAGE_SIZE, cfg.HEIGHT_CROP_OFFSET) # train_data_generator use a yield keyword, thus this is a generator

Preparing data... batch_size: 2, image_size: (1536, 512), crop_offset: 690
(21914, 2)


In [ ]:
images, labels = next(data_generator)
images = images.to(device)
labels = labels.to(device)
print(images.shape, labels.shape)

out = net(images)
print(out.shape)

optimizer.zero_grad()

loss, mean_iou = utils.create_loss(out, labels, NUM_CLASSES)
print(loss, mean_iou)

loss.backward()
optimizer.step()

In [ ]:
loss.item(), mean_iou.item()

In [ ]:
plt.imshow(labels.cpu().numpy()[0])

In [ ]:
plt.imshow(out.cpu().detach().numpy()[0][6])

In [ ]:

out.cpu().detach().numpy()[0][0]

In [ ]:

out.cpu().detach().numpy()[0][3]

In [ ]:
np.sum(np.argmax(out.cpu().detach().numpy(), axis=1))

In [ ]:
# sys.path.append('/home/howard/Nutstore Files/我的坚果云/code_tools')

In [ ]:
# from pyhw import hwimage

In [ ]:
img = out.detach().numpy()
print(img.shape)
# plt.imshow(img[0][1])
imgs = []
for i in range(0, 3):
    imgs.append(img[0][i])
hwimage.imshow_multi(imgs)

In [ ]:
imgs = []
for i in range(3, 6):
    imgs.append(img[0][i])
hwimage.imshow_multi(imgs)

In [ ]:
imgs = []
for i in range(6, 8):
    imgs.append(img[0][i])
hwimage.imshow_multi(imgs)